In [1]:
import re
import tarfile
import functools

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import spacy 
nlp = spacy.load('en')

In [3]:
def tokenize(sent):
    tok_sent = nlp(sent)
    pos_list = []
    text_list = []
    for s in tok_sent:
        pos_list.append(s.pos)
        test_list.append(s.pos_)
    
    return (test_list, pos_list)

In [4]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    story_pos = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
            story_pos = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
                substory_pos = [x for x in story if x]
            data.append((substory, substory_pos, q, subpos, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
            story_pos.append(sent)
    return data

In [5]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, flatten(story_pos), subpos, answer) 
            for story, q, story_pos, subpos, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [7]:
#FIXME! 
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xspos = []
    xqs = []
    xqspos = []
    ys = []
    ypos = []
    for story, story_pos, query, answer in data:
        x = [word_idx[w] for w in story]
        xspos = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        xqspos = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(x)
        xqs.append(xq)
        xqpos.append(xq)
        ys.append(y)
        ypos.append(ypos)
    return pad_sequences(xs, maxlen=story_maxlen),(xspos, maxlen=story_maxlen),
           pad_sequences(xqs, maxlen=query_maxlen), pad_sequences(xqpos, maxlen=query_maxlen), 
           np.array(ys), np.array(ypos)

SyntaxError: invalid syntax (<ipython-input-7-ba4615ef8cb5>, line 22)

In [ ]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 4
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

In [ ]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [ ]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

In [ ]:
vocab = set()
for story, q, answer in train + test:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

In [ ]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

In [ ]:
x, xq, y = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)
#fixme
#xspos, xqpos, ypos = vectorize_stories(train, word_idx, story_maxlen, query_maxlen)
#txpos, txqpos, typos = vectorize_stories(test, word_idx, story_maxlen, query_maxlen)

In [ ]:
print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))

#FIXME
'''
print('xspos.shape = {}'.format(xspos.shape))
print('xqpos.shape = {}'.format(xqpos.shape))
print('ypos.shape = {}'.format(ypos.shape))
'''
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

In [ ]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') #Input the story as a sequence of tokens
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence) #Embed the words in the story
encoded_sentence = layers.Dropout(0.3)(encoded_sentence) #Dropout regularization over word vectors

question = layers.Input(shape=(query_maxlen,), dtype='int32') #Input the question as a sequence of tokens
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question) #Embed the words in the story
encoded_question = layers.Dropout(0.3)(encoded_question) #Dropout regularization over word vectors
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question) #Feed the question tokens through an LSTM
#Generate 50-dim feature vector at end, line up 50-dim feature vector with each token in input
encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)

merged = layers.add([encoded_sentence, encoded_question]) #Combine lined up question feature vector and embeddings
merged = RNN(EMBED_HIDDEN_SIZE)(merged) #Feed combined 100-dim features through an LSTM
#Generate 50-dim feature vector at end
merged = layers.Dropout(0.3)(merged) #Dropout regularization over 50-dim feature vector
#Classify feature vector -> vocab word to find answer
preds = layers.Dense(vocab_size, activation='softmax')(merged) 


model = Model([sentence, question], preds) #Creates model with inputs of sentence and quetion, output is pred
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) #Compiles entire model for training 

In [ ]:
print('Training')
model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))
